In [1]:
import planetary_computer
import pystac_client
import odc.stac
import xarray as xr
import rioxarray
import geopandas as gpd
from datetime import datetime
import os

# Define output directory
output_dir = "/home/george/geospatial_workflows_automation/geospatial_datacube/data"
os.makedirs(output_dir, exist_ok=True)

In [2]:
import planetary_computer
import pystac_client
import xarray as xr

# Connect to Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1", 
    modifier=planetary_computer.sign_inplace,
)

# Define dataset collections
collections = {
    "temperature": "sentinel-3-slstr-lst-l2-netcdf",
    "precipitation": "terraclimate",
    "ndvi": "sentinel-2-l2a",
    "elevation": "cop-dem-glo-30",
    "sentinel_1": "sentinel-1-grd", 
    "Fire": "sentinel-3-slstr-frp-l2-netcdf"
}

# Define the time range
time_range = "2024-01-01/2024-12-31"

# Define area of interest (Modify with your actual bounding box)
aoi = {
    "west": 33.9, "south": 0.1, "east": 34.6, "north": 0.8  # Example for Busia, Kenya
}

# Query each dataset
for name, collection in collections.items():
    search = catalog.search(
        collections=[collection],
        datetime=time_range,
        bbox=[aoi["west"], aoi["south"], aoi["east"], aoi["north"]]
    )
    items = list(search.item_collection())
    print(f"{name}: Found {len(items)} items in {collection}")

temperature: Found 187 items in sentinel-3-slstr-lst-l2-netcdf
precipitation: Found 0 items in terraclimate
ndvi: Found 292 items in sentinel-2-l2a
elevation: Found 0 items in cop-dem-glo-30
sentinel_1: Found 102 items in sentinel-1-grd
Fire: Found 865 items in sentinel-3-slstr-frp-l2-netcdf


In [3]:
api_url = 'https://earth-search.aws.element84.com/v1'
client = pystac_client.Client.open(api_url)
for collection in client.get_collections():
    print(collection)

<CollectionClient id=sentinel-2-pre-c1-l2a>
<CollectionClient id=cop-dem-glo-30>
<CollectionClient id=naip>
<CollectionClient id=cop-dem-glo-90>
<CollectionClient id=landsat-c2-l2>
<CollectionClient id=sentinel-2-l2a>
<CollectionClient id=sentinel-2-l1c>
<CollectionClient id=sentinel-2-c1-l2a>
<CollectionClient id=sentinel-1-grd>


In [4]:
# Search Copernicus DEM (Elevation Data)
dem_search = client.search(
    collections=["cop-dem-glo-30"],  # Global 30m resolution DEM 
    bbox=[aoi["west"], aoi["south"], aoi["east"], aoi["north"]],
)

# Convert search results to a list
dem_items = list(dem_search.items())

# Print number of DEM items found
print(f"Elevation: Found {len(dem_items)} items in cop-dem-glo-30")

Elevation: Found 2 items in cop-dem-glo-30


In [5]:
# Define dataset collections
collections = {
    "sentinel_1": "sentinel-1-grd",
    # "elevation": "cop-dem-glo-30",
    "ndvi": "sentinel-2-l2a"
}

# Define the time range
time_range = "2024-01-01/2024-12-31"

# Define area of interest (Modify with your actual bounding box)
aoi = {
    "west": 33.9, "south": 0.1, "east": 34.6, "north": 0.8  # Example for Busia, Kenya
}

# Query each dataset
for name, collection in collections.items():
    search = client.search(
        collections=[collection],
        datetime=time_range,
        bbox=[aoi["west"], aoi["south"], aoi["east"], aoi["north"]]
    )
    items = list(search.item_collection())
    print(f"{name}: Found {len(items)} items in {collection}")

print(f"Elevation: {dem_items}")

sentinel_1: Found 111 items in sentinel-1-grd
ndvi: Found 295 items in sentinel-2-l2a
Elevation: [<Item id=Copernicus_DSM_COG_10_N00_00_E034_00_DEM>, <Item id=Copernicus_DSM_COG_10_N00_00_E033_00_DEM>]


In [6]:
import ee

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Define Area of Interest (Busia, Kenya)
aoi = ee.Geometry.BBox(33.9, 0.1, 34.6, 0.8)

# Load the SRTM elevation dataset
elevation = ee.Image("USGS/SRTMGL1_003").clip(aoi)

# Get elevation statistics
stats = elevation.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

print("Mean Elevation:", stats.getInfo())

Mean Elevation: {'elevation': 1250.3874471512086}


In [7]:
# Search for relevant datasets in GEE
import ee
import warnings
warnings.simplefilter("ignore", category=UserWarning)

# Initialize GEE
ee.Initialize()

# Define datasets
datasets = {
    "precipitation": "UCSB-CHG/CHIRPS/DAILY",
    "sentinel_2": "COPERNICUS/S2_SR_HARMONIZED",
    "sentinel_1": "COPERNICUS/S1_GRD"
}

# Define AOI (Example: Busia, Kenya)
aoi = ee.Geometry.Rectangle([33.9, 0.1, 34.6, 0.8])

# Define date range
start_date = "2024-01-01"
end_date = "2024-12-31"

# Search each dataset
for name, dataset in datasets.items():
    collection = ee.ImageCollection(dataset).filterDate(start_date, end_date).filterBounds(aoi)
    count = collection.size().getInfo()
    print(f"{name}: Found {count} items in {dataset}")

precipitation: Found 365 items in UCSB-CHG/CHIRPS/DAILY
sentinel_2: Found 298 items in COPERNICUS/S2_SR_HARMONIZED
sentinel_1: Found 111 items in COPERNICUS/S1_GRD
